<h1><strong>Searching Google for Machine Learning PDFs</strong></h1>   
<img src="GoogleSearch.png">

<strong>By:</strong> Jeff Ocampo   
<strong>Created:</strong> May 2020   

<h3><strong>About</strong></h3>
<strong>Q:</strong> What is this doing?   

<strong>A:</strong> Searching Google for machine learning PDFs and creating a database of links   

<h3><strong>Steps:</strong></h3>
<ol>
    <li><a href="https://developers.google.com/custom-search/v1/overview">Set up Google Custom Search JSON API</a></li>
    <li>Set up SQL database for results</li>
    <li>Python for:</li>
    <ul>
        <li>API call</li>
        <li>JSON parse</li>
        <li>Insert into SQL tables</li>
    </ul>
</ol>   

<h3><strong>SQL Server Setup</strong></h3>   
Use of a SQL database isn't necessary for this project, but is worthwhile if I decide to scale this.   

Also, in some ways it is more convenient and organized than working with flat files or other storage methods.   
<img src="TableCreation.png">   

<h3><strong>Final Results in SQL Tables</strong></h3>   
After looping over each page of the search results and parsing out the links, titles, and other information, two tables are populated on the server.   
The first table is a search history table with some metadata about the search, include what page and number of results.    
The second table is populated with search results.   
Google limits Custom Search Results to 100, so that's the max results for a specific set of parameters.   
I only show results for the search term "machine learning" and the PDF file type.   
<img src="SearchHist.png">
<img src="SearchResults.png">   

<h3><strong>Python Code</strong></h3>   
Below is the code used once the Google Custom Search and SQL Server tables were configured.   


<strong>Packages</strong>

In [80]:
import requests
import getpass
import json
import pyodbc
import datetime

<br>
<strong>Variable search term and file type</strong>

In [118]:
search_term = 'machine learning'
search_file_type = 'PDF'

<br>
<strong>Other settings</strong>

In [173]:
# Hey friend, don't think you need to know my API Key :)
api_key = getpass.getpass("Key: ")

Key:  ·······································


In [3]:
# Hey friend, don't think you need to know this either :)
custom_search_id = getpass.getpass("Custom search: ")

Custom search:  ·································


In [66]:
# Hey, I just met you, and this is crazy, but here's my server name, so call me, maybe?
# ...not showing you this either
sql_server = getpass.getpass('SQL Sever name: ')

SQL Sever name:  ···············


In [73]:
sql_conn = pyodbc.connect('Driver={SQL Server};Server='+sql_server+';Database=GOOGLE_FILES;Trusted_Connection=yes;')
sql_cursor = sql_conn.cursor()

<br>
<strong>Functions</strong>

In [175]:
def req_stat(code):
    """
    Simply decodes and prints the requests status code
    """
    if code == 200:
        print('Request successful')
    elif code == 301:
        print('Request redirected')
    elif code == 401:
        print('Request failed due to authentication')
    elif code == 400:
        print('Request failed due to bad request')
    elif code == 403:
        print('Request failed, forbidden')
    elif code == 404: 
        print('Reques failed, not found')
    else:
        print('Other request status code')


In [176]:
def get_search_results(search_url, search_params_dic):
    """
    Make the API call and return JSON data
    """
    search_response = requests.get(search_url, params=search_params_dic)
    #req_stat(search_response.status_code)
    if search_response.status_code == 200:
        search_json_data = search_response.json()
    
    return search_json_data
    

In [177]:
def search_item_count(json_data):
    """
    Returns the number of items in the JSON data
    """
    item_count = len( json_data["items"])
    
    return item_count

In [178]:
def extract_data(json_data):
    """
    For the JSON data from the request, return data in list format for insertion into SQL Server
    """
    extract_dt = datetime.datetime.now()
    results_list = []
    for item in json_data["items"]:
        title = item["title"]
        link = item["link"]
        snippet = item["snippet"]
        try:
            pagemap = json.dumps(item["pagemap"])
        except:
            pagemap = ''
        results_list.append([title, link, snippet, pagemap, extract_dt])
    
    return results_list


In [179]:
def prep_hist_data(json_data, search_term, search_file_type, api_parm, item_cnt, search_dt):
    """
    Get data for search hist table insert, including results count
    """
    search_hits = json_data["searchInformation"]["totalResults"]
    
    hist_data = [search_term, search_file_type, api_parm['start'], item_cnt, search_dt,search_hits]
    
    return hist_data
    

In [180]:
def sql_hist_insert(hist_list):
    """
    Single line insert into the SEACH_HIST table
    """
    search_hist_query = """INSERT INTO DBO.SEARCH_HIST(SEARCH_TERM, SEARCH_FILETYPE, SEARCH_START, SEARCH_ITEM_CNT, SEARCH_DT, SEARCH_RESULTS_CNT) 
    VALUES (?,?,?,?,?,?)
    """
    sql_cursor.execute( search_hist_query, hist_list)
    sql_conn.commit()

In [181]:
def sql_results_insert(results_list):
    """
    Multiple line insert into the SEARCH_RESULTS table
    """
    search_results_query = """INSERT INTO DBO.SEARCH_RESULTS(TITLE,LINK,SNIPPET,PAGEMAP,ENTRY_DATE) 
    VALUES (?,?,?,?,?)
    """
    sql_cursor.executemany( search_results_query, results_list)
    sql_conn.commit()

In [182]:
def main_search(search_term, search_file_type):
    """
    Main function that executes the search and processing into the SQL tables
    Iterates over the first 100 (according to https://developers.google.com/custom-search/v1/reference/rest/v1/cse/list#query-parameters, only the first 100 will ever be returned)
    """
    search_dt = datetime.datetime.now()
    search_url = 'https://customsearch.googleapis.com/customsearch/v1'
    api_parm = {"key": api_key, "cx": custom_search_id, "q": search_term, "fileType": search_file_type, "start": 1}
    
    start_list = list(range(1,92,10))

    for i in start_list:
        
        api_parm["start"] = i
        
        #print('Making request...')
        tmp_json_data = get_search_results(search_url, api_parm)

        #print('Parsing data...')
        tmp_item_cnt = search_item_count(tmp_json_data)
        if tmp_item_cnt > 0:
            tmp_results_list = extract_data(tmp_json_data)

        #print('Formatting search history data...')
        tmp_hist_data = prep_hist_data( tmp_json_data, search_term, search_file_type, api_parm, tmp_item_cnt, search_dt)

        #print('Saving search history...')
        sql_hist_insert(tmp_hist_data)

        #print('Saving search results...')
        if tmp_item_cnt > 0:
            sql_results_insert(tmp_results_list)
    

<br>
<strong>Execute!</strong>

In [183]:
main_search(search_term, search_file_type)